In [ ]:
%run includes/shared-setup.ipynb

In [ ]:
org_id = widgets.Text(
    placeholder='12345',
    description='GCP Org ID',
    disabled=False
)
display(org_id)

In [ ]:
# res heiracrchy functions
def get_icon(resType):
    if resType == "project":
        return 'codepen'
    else:
        return resType

def build_tree_node(df, parent_name, parent_node=None):
    children = df[df['parentDisplayName'] == parent_name]
    
    for _, child in children.iterrows():
        child_node = Node(child['displayName'], opened=False, icon=get_icon(child['resType']))
        if parent_node:
            parent_node.add_node(child_node)
        build_tree_node(df, child['displayName'], child_node)

In [ ]:
def query_and_format(entity_id, query_fn, parent_display_name, res_type):
    df = run_stackql_query(query_fn(entity_id))
    df["parentDisplayName"] = parent_display_name
    df["resType"] = res_type
    return df

def get_resources_recursive(entity_id, get_projects_query_fn, get_folders_query_fn, parent_display_name='organization'):
    dfs = []  # List to store DataFrames

    with ThreadPoolExecutor() as executor:
        future_project = executor.submit(query_and_format, entity_id, get_projects_query_fn, parent_display_name, "project")
        future_folder = executor.submit(query_and_format, entity_id, get_folders_query_fn, parent_display_name, "folder")

        projects_df = future_project.result()
        print_overwrite(f"Found {len(projects_df)} projects in {entity_id}")
        dfs.append(projects_df)

        folders_df = future_folder.result()
        print_overwrite(f"Found {len(folders_df)} folders in {entity_id}")
        dfs.append(folders_df)

        # Parallelize the fetching of child resources
        folder_futures = [executor.submit(get_resources_recursive, folder['name'], get_projects_query_fn, get_folders_query_fn, folder['displayName']) 
                          for _, folder in folders_df.iterrows() if 'name' in folder]
        
        for future in as_completed(folder_futures):
            dfs.append(future.result())

    # Concatenate all collected DataFrames at once
    resources_df = pd.concat(dfs, ignore_index=True)
    return resources_df

def get_all_resources(get_projects_query, get_folders_query, org_id, show_time=True):
    start_time = time.time()
    
    # Start with the root organization to get all resources
    resources_df = get_resources_recursive("organizations/%s" % (org_id), get_projects_query, get_folders_query)
    
    # Create root node and build the tree
    root = Node("organization", opened=False, icon='building')
    build_tree_node(resources_df, "organization", root)
    
    # Display the tree
    tree = Tree(nodes=[root])
    
    # Calculate metrics and display
    elapsed_time = round(time.time() - start_time)
    num_folders = resources_df.query("resType == 'folder'").shape[0]
    num_projects = resources_df.query("resType == 'project'").shape[0]
    all_projects = resources_df.query("resType == 'project'")['projectId'].dropna().tolist()
    all_project_numbers = [int(item.split('/')[-1]) for item in resources_df['name'] if item.startswith('projects/')]
    
    cards_data = [("Number of Projects", num_projects), ("Number of Folders", num_folders)]

    resources_df.to_pickle('/tmp/resources_df.pkl')

    with open('/tmp/all_projects.pkl', 'wb') as f:
        pickle.dump(all_projects, f)

    with open('/tmp/all_project_numbers.pkl', 'wb') as f:
        pickle.dump(all_project_numbers, f)

    clear_output(wait=True)
    if show_time:
        print(f"Total elapsed time: {elapsed_time} seconds")

    return tree, cards_data

In [ ]:
def plot_serviceusage():
    all_services = [
        ('servicenetworking', servicenetworking_projects),
        ('compute', compute_projects),
        ('storage', storage_projects),
        ('composer', composer_projects),
        ('bigquery', bigquery_projects),
        ('logging', logging_projects),
        ('dialogFlow', dialogflow_projects),
        ('cloudfunctions', cloudfunctions_projects),
        ('contactcenteraiplatform', contactcenteraiplatform_projects),
        ('monitoring', monitoring_projects),
        ('datastudio', datastudio_projects),
        ('notebooks', notebooks_projects),
        ('aiplatform', aiplatform_projects),
        ('artifactregistry', artifactregistry_projects),
        ('containerregistry', containerregistry_projects),
    ]
    
    # Count the number of projects for each service
    service_counts = [(service[0], len(service[1])) for service in all_services]
    
    # Sort by the number of projects, most to least
    service_counts.sort(key=lambda x: x[1], reverse=True)
    
    # Extract sorted service names and counts
    sorted_service_names = [x[0] for x in service_counts]
    sorted_project_counts = [x[1] for x in service_counts]
    
    # Create the bar chart
    fig = go.Figure(data=[go.Bar(x=sorted_service_names, y=sorted_project_counts)])
    
    # Add titles and labels
    fig.update_layout(title='Enabled GCP Services',
                      xaxis_title='GCP Service',
                      yaxis_title='Number of Projects')
    
    # Show the figure
    fig.show()